In [10]:
import sqlite3
import pandas as pd

from google.cloud import bigquery
from google.oauth2 import service_account


In [2]:
# JSON key location
service_path = "C:/Users/vanes/OneDrive/Desktop/Work/MSBA/ADA/wedge_project/"
service_file = 'wedge-project-vw-key.json'  

# Give my credentials to Google
credentials = service_account.Credentials.from_service_account_file(service_path + service_file)

# GBQ project ID and dataset ID
proj_id = 'wedge-project-vw'

# And finally we establish our connection
client = bigquery.Client(credentials = credentials, project=proj_id)


In [11]:
# Connect to a database (or create it if it doesn't exist)
conn = sqlite3.connect('wedge_data2.db')

# Create a cursor object to interact with the database
cursor = conn.cursor()

In [12]:
#1.	Sales by date by hour: By calendar date (YYYY-MM-DD) and hour of the day, 
# determine the total spend in the store, the number of transactions, 
# and a count of the number of items . 

# Write the query
query_1 = """SELECT EXTRACT(DATE FROM datetime) as date, 
       EXTRACT(HOUR from datetime) as hour,
       SUM(total) as spend,
       COUNT(DISTINCT trans_id) as trans,
       COUNT(description) as items
FROM `wedge-project-vw.wedge_data2.transArchive_*` 
GROUP BY date, hour
ORDER BY date, hour ASC
;"""

# Execute the query
query_job = client.query(
    query_1,
    location="US",
)

# Convert the result to a dataframe
sales_date_hour = query_job.to_dataframe()

# Write the dataframe to a SQLite table
sales_date_hour.to_sql('sales_date_hour', conn, index=False, if_exists='replace')



39421

In [13]:
#2.	Sales by owner by year by month: A file that has the following columns: 
# card_no, year, month, sales, transactions, and items. 

# Write the query
query_2 = """SELECT card_no,
       EXTRACT(YEAR FROM datetime) as year, 
       EXTRACT(MONTH from datetime) as month,
       SUM(total) as sales,
       COUNT(DISTINCT trans_id) as trans,
       COUNT(description) as items
FROM `wedge-project-vw.wedge_data2.transArchive_*` 
GROUP BY card_no, year, month
ORDER BY year, month ASC
;"""

# Execute the query
query_job2 = client.query(
    query_2,
    location="US",
)

# Write the query results to a dataframe
owner_year_month = query_job2.to_dataframe()

# Write the dataframe to SQLite
owner_year_month.to_sql('owner_year_month', conn, index=False, if_exists='replace')

810208

In [14]:

#3.	Sales by product description by year by month: A file that has the following columns:
# upc, description, department number, department name, year, month, sales, transactions, and items.

# Query for third prompt
query_3 = """SELECT upc,
       description,
       trans.department, 
       dl.dept_name, 
       EXTRACT(YEAR FROM datetime) as year, 
       EXTRACT(MONTH from datetime) as month,
       SUM(total) as sales,
       COUNT(DISTINCT trans_id) as trans,
       COUNT(description) as items
FROM `wedge-project-vw.wedge_data2.transArchive_*` trans
JOIN `wedge_data2.dept_lookup` dl on  trans.department = dl.department
GROUP BY upc, description, department, dept_name, year, month
ORDER BY year, month ASC
;"""

# Execute the query
query_job3 = client.query(
    query_3,
    location="US",
)

# Convert results of query to dataframe
product_year_month = query_job3.to_dataframe()

# Create SQLite table
product_year_month.to_sql('product_year_month', conn, index=False, if_exists='replace')

1214096

In [15]:
# Commit the changes and close the connection
conn.commit()
conn.close()